In [4]:
import json
import boto3

In [5]:
f = open('./credentials.json', 'r')
credentials = json.loads(f.read())
f.close()

In [6]:
s3 = boto3.resource('s3', 
                    aws_access_key_id=credentials['AccessKeyID'], 
                    aws_secret_access_key=credentials['SecretAccessKey'])

In [10]:
# Create the bucket
BUCKET_NAME = 'ningqiwbuckethw3'
LOCATION = 'us-east-2'
s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration={ 'LocationConstraint': LOCATION })

s3.Bucket(name='ningqiwbuckethw3')

In [19]:
dyndb = boto3.resource('dynamodb', 
                       region_name=LOCATION,
                       aws_access_key_id=credentials['AccessKeyID'], 
                       aws_secret_access_key=credentials['SecretAccessKey'])

In [21]:
# Create the table to store the data
TABLE_NAME = 'ExpData'
table = dyndb.create_table(
    TableName=TABLE_NAME,
    KeySchema=[
        { 'AttributeName': 'PartitionKey', 'KeyType': 'HASH' },
        { 'AttributeName': 'RowKey',       'KeyType': 'RANGE' },
    ],
    AttributeDefinitions=[
        { 'AttributeName': 'PartitionKey', 'AttributeType': 'S' },
        { 'AttributeName': 'RowKey',       'AttributeType': 'S' },
    ],
    BillingMode='PROVISIONED',
    ProvisionedThroughput={
        'ReadCapacityUnits': 128,
        'WriteCapacityUnits': 128
    },
)
# table = dyndb.Table(TABLE_NAME)

In [23]:
table.meta.client.get_waiter('table_exists').wait(TableName=TABLE_NAME)

In [24]:
import csv
import os

In [30]:
URL_BASE = f'https://s3-{LOCATION}.amazonaws/com/{BUCKET_NAME}/'
DATA_DIR = './HW3csv/'

with open(os.path.join(DATA_DIR, 'experiments.csv'), 'r') as csv_file:
    items = csv.reader(csv_file, delimiter=',', quotechar='|')
    next(items) # Skip the header
    for item in items:
        body = open(os.path.join(DATA_DIR, item[4]), 'rb')
        s3.Object(BUCKET_NAME, item[4]).put(Body=body)
        s3.Object(BUCKET_NAME, item[4]).Acl().put(ACL='public-read')
        
        url = URL_BASE + item[4]
        metadata_item = {
            'PartitionKey': item[0],
            'RowKey': item[1],
            'Conductivity': item[2],
            'Concentration': item[3],
            'URL': url,
        }
        table.put_item(Item=metadata_item)

In [37]:
from boto3.dynamodb.conditions import Key

In [47]:
# Query the table we created just now
db_client = boto3.client('dynamodb', 
                         region_name=LOCATION,
                         aws_access_key_id=credentials['AccessKeyID'], 
                         aws_secret_access_key=credentials['SecretAccessKey'])

query_resp = db_client.query(
    TableName=TABLE_NAME,
    KeyConditionExpression="PartitionKey = :id",
    ExpressionAttributeValues={
        ":id": {'S': '1'},
    }
)
print(query_resp['Items'])

[{'Concentration': {'S': '3.4'}, 'PartitionKey': {'S': '1'}, 'RowKey': {'S': '-1'}, 'Conductivity': {'S': '52'}, 'URL': {'S': 'https://s3-us-east-2.amazonaws/com/ningqiwbuckethw3/exp1.csv'}}]
